## Lab 5 ##
Kar Lok Ng

8971216

In [80]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, get_scorer_names


X, y = datasets.load_diabetes(as_frame=True, scaled=False, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [81]:
deg_arr = np.arange(0,9)

# the i-th element of the list is the mean score or std of the i-th degree model 
mean_train_r2 = []
mean_train_mae = []
mean_train_mape = []
std_train_r2 = []
std_train_mae = []
std_train_mape = []

for deg in deg_arr:
    model = make_pipeline(PolynomialFeatures(deg), LinearRegression())

    # we note that cross_val_score by default uses LinearRegression's 'score' method
    # which is the R^2 value of that model
    # for clarity I've specified for scoring to use 'r2'
    score_mean_r2 = cross_val_score(model, X_train, y_train, scoring= 'r2', cv=5).mean()
    score_std_r2 = cross_val_score(model, X_train, y_train, scoring= 'r2', cv=5).std()
    mean_train_r2.append(score_mean_r2)
    std_train_r2.append(score_std_r2)

    score_mean_mae = cross_val_score(model, X_train, y_train, scoring= "neg_mean_absolute_error",cv=5).mean()
    score_std_mae = cross_val_score(model, X_train, y_train, scoring= "neg_mean_absolute_error",cv=5).std()
    mean_train_mae.append(-1*score_mean_mae)
    std_train_mae.append(score_std_mae)

    score_mean_mape = cross_val_score(model, X_train, y_train, scoring= "neg_mean_absolute_percentage_error",cv=5).mean()
    score_std_mape = cross_val_score(model, X_train, y_train, scoring= "neg_mean_absolute_percentage_error",cv=5).std()
    mean_train_mape.append(-1*score_mean_mape)
    std_train_mape.append(score_std_mape)


In [82]:
col_names = ['deg_' + str(deg) for deg in deg_arr]
row_names = ['mean_r2', 'std_r2', 'mean_mae', 'std_mae', 'mean_mape', 'std_mape']
values = [mean_train_r2, std_train_r2, mean_train_mae, std_train_mae, mean_train_mape, std_train_mape]
metrics_train_df = pd.DataFrame(data= values, columns= col_names, index= row_names)
display(metrics_train_df)

,deg_0,deg_1,deg_2,deg_3,deg_4,deg_5,deg_6,deg_7,deg_8
mean_r2,-0.039901,0.452281,0.117167,-1308.500045,-170.278504,-336.772695,-1144.363125,-3723.148855,-12124.058339
std_r2,0.048473,0.127329,0.367606,720.541644,93.913729,378.566338,1406.769177,4575.033673,15563.949357
mean_mae,67.170145,45.580788,55.410843,1338.668658,504.097826,537.839909,744.366967,1085.011170,1661.694072
std_mae,7.652792,1.609128,4.602439,119.467827,124.844577,235.187940,386.767659,639.537525,1145.712887
mean_mape,0.641334,0.408418,0.487661,10.593700,3.860945,3.905800,5.711166,8.638523,13.635322
std_mape,0.079557,0.049973,0.068001,1.852179,0.909767,1.939440,3.495901,6.149951,11.408246


3) From the table, the best model seems to be the model with degree 1. 

    It has the highest mean $R^2$ value, which means that, in the 5 fold cross-validation, it has the highest average proportion of variance explained by the model. It also posses the lowest mean MAE and mean MAPE, thus it has the lowest difference between the predicted value and the true value. Furthermore, it has a relatively small standard deviation within the cross validation for all 3 values, thus it performs relatively consistently when different chunks of the data are taken out for validation. 

    Thus, the best model as judged by these 3 metrics would be the model of degree 1. 

4) We can fit the model on the whole training data, and look at the coefficients of the fitted model and performance on the test data to see if we can gather any further insights. 

In [113]:
model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

train_r2 = r2_score(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)

print("")
print('Train metrics: ')
print(train_r2, train_mae, train_mape)
print('')
print('Test metrics: ')
print(test_r2, test_mae, test_mape)

#coef_series = pd.Series(dict(zip(X_train.columns, model.coef_)))
#print(coef_series)


Train metrics: 
0.5244124363545944 44.09783499263568 0.3959249325786459

Test metrics: 
0.4772897164322617 41.91937845679274 0.3667196318312674


We can see that the performance metrics between the training and test data is not too dissimilar, and thus I do not think overfitting is currently an issue.

We can take a look if our independent variables are correlated.

In [116]:
from scipy.stats import pearsonr
def pearsonr_pval(X, y):
    return pearsonr(X, y)[1]

X.corr(method=pearsonr_pval)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
age,1.000000e+00,2.423425e-04,9.076792e-05,4.392569e-13,2.894154e-08,3.271391e-06,1.144857e-01,1.568788e-05,7.251191e-09,9.347961e-11
sex,2.423425e-04,1.000000e+00,6.404796e-02,2.922214e-07,4.594318e-01,2.649987e-03,1.486840e-16,7.667030e-13,1.573571e-03,1.025312e-05
bmi,9.076792e-05,6.404796e-02,1.000000e+00,5.413797e-18,1.032152e-07,2.514909e-08,1.594491e-15,1.031196e-19,5.237528e-23,2.170657e-17
bp,4.392569e-13,2.922214e-07,5.413797e-18,1.000000e+00,2.465826e-07,8.708610e-05,1.580418e-04,3.918683e-08,8.089411e-18,1.517459e-17
s1,2.894154e-08,4.594318e-01,1.032152e-07,2.465826e-07,1.000000e+00,8.341807e-158,2.797935e-01,3.819722e-35,2.247028e-31,2.206445e-12
s2,3.271391e-06,2.649987e-03,2.514909e-08,8.708610e-05,8.341807e-158,1.000000e+00,3.194540e-05,1.410336e-56,7.220422e-12,4.751211e-10
s3,1.144857e-01,1.486840e-16,1.594491e-15,1.580418e-04,2.797935e-01,3.194540e-05,1.000000e+00,2.484665e-77,2.786014e-18,4.917718e-09
s4,1.568788e-05,7.667030e-13,1.031196e-19,3.918683e-08,3.819722e-35,1.410336e-56,2.484665e-77,1.000000e+00,6.976741e-48,4.820011e-20
s5,7.251191e-09,1.573571e-03,5.237528e-23,8.089411e-18,2.247028e-31,7.220422e-12,2.786014e-18,6.976741e-48,1.000000e+00,4.666132e-25
s6,9.347961e-11,1.025312e-05,2.170657e-17,1.517459e-17,2.206445e-12,4.751211e-10,4.917718e-09,4.820011e-20,4.666132e-25,1.000000e+00


We can see that there are several of these independent variables that are statistically significantly correlated to one another, such as bmi and bp with p_value at $5.41 \times 10^{-8}$. In OLS, there is an assumption that the columns of independent variable are independent of each other (as otherwise, ). This assumption is violated, and thus there may be a collinearity problem in our dataset, which would make interpretation coefficients more difficult to interpret (as the variance may be incredibly high). 